In [1]:
import mysql.connector
from getpass import getpass

In [2]:
password = getpass()

In [3]:
#1. Establish a connection between Python and the Sakila database.
conn = mysql.connector.connect(host = "localhost",
                               user = "root",
                               password = password,
                               database = "sakila")

In [4]:
#cheking if connection is successfull
conn.is_connected()

True

In [5]:
#now are creating a cursor, that will hole our query, execute it, and fetch my results -object that go from python to sql
cursor = conn.cursor()

In [11]:
#2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:
##engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
##month: an integer representing the month for which rental data is to be retrieved.
##year: an integer representing the year for which rental data is to be retrieved.

def rentals_month (engine, month, year):
    query = f"""
            SELECT 
                rental_id,
                rental_date,
                inventory_id,
                customer_id,
                return_date,
                staff_id
            FROM 
                rental
            WHERE 
                MONTH(rental_date) = {month} AND
                YEAR(rental_date) = {year}
            """
    rental_data = pd.read_sql(query, con=engine)

    return rental_data



In [13]:
from sqlalchemy import create_engine #import library
from sklearn.datasets import load_iris

In [20]:
#3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.
def rental_count_month(df, month, year):
    df.rename(columns={f'rentals_{month}_{year}': f'rentals_{month:02d}_{year}'}, inplace=True)
    return df

In [21]:
#4. 
def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    combined_df['difference'] = combined_df.iloc[:, 1] - combined_df.iloc[:, 2]
    return combined_df